In [70]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime 
import re

## Data preparation

In [71]:
output_dir = '../output_data'
file_name = 'cleaned_data_02.csv'

# Read the cleaned data
df = pd.read_csv(os.path.join(output_dir, file_name))
df.head()

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time
0,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:50
1,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,.,ls .,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54
2,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54
3,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,..,ls ..,NaN,jovyan,2023-03-23:20:54
4,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan/data-store/data/input/RNA_SEQ_...,NaN,Drug_A\nDrug_B,2023-03-23:20:54


In [72]:
df

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time
0,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:50
1,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,.,ls .,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54
2,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54
3,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,..,ls ..,NaN,jovyan,2023-03-23:20:54
4,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan/data-store/data/input/RNA_SEQ_...,NaN,Drug_A\nDrug_B,2023-03-23:20:54
...,...,...,...,...,...,...,...,...,...,...
1540,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,cd,NaN,NaN,cd,NaN,NaN,2023-12-21:21:33
1541,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which pwd,NaN,/usr/bin/pwd,2023-12-21:21:36
1542,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which cd,NaN,NaN,2023-12-21:21:36
1543,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which which,NaN,/usr/bin/which,2023-12-21:21:36


In [81]:
# Initialize lists for the new dataframe
error_list = []
suggestion_list = []

# Iterate through the dataframe
for i in range(len(df)):
    # If 'stderr' is not NaN/None, add its value and the subsequent 'command' value to the new lists
    if pd.notna(df['stderr'][i]):
        error_list.append(df['stderr'][i])
        if i + 1 < len(df):
            suggestion_list.append(df['command'][i + 1])
        else:
            suggestion_list.append(None)

# Create a new dataframe with the two columns
paired = pd.DataFrame({
    'error': error_list,
    'suggestion': suggestion_list
})

error_suggestion = paired.groupby('error')['suggestion'].agg(lambda x: x.value_counts().idxmax()).reset_index()

In [82]:
error_suggestion

,error,suggestion
0,/bin/ls: cannot access 'RNA_SEQ_SAMPLES': No s...,/usr/bin/which
1,/home/jovyan/.offline.sh: eval: line 354: unex...,cd
2,/home/jovyan/.offline.sh: eval: line 368: unex...,echo
3,/home/jovyan/.offline.sh: line 354: cd: ...: N...,cd
4,/home/jovyan/.offline.sh: line 354: cd: ./../D...,cd
...,...,...
84,ls: invalid option -- '.'\nTry 'ls --help' for...,cd
85,mkdir: cannot create directory ‘bin’: Permissi...,ls
86,mkdir: cannot create directory ‘misc’: Permiss...,ls
87,mkdir: cannot create directory ‘mschild’: Perm...,ls


In [83]:
folder = '../output_data/'
output_file_path = 'error_suggestion.csv'

# Export the dataframe to CSV
error_suggestion.to_csv(folder+output_file_path, index=False)

print(f"DataFrame exported to {output_file_path}")

DataFrame exported to error_suggestion.csv


In [4]:
errors = df.copy()
# Drop rows where the value in stderr column is NaN
errors = errors.dropna(subset=['stderr'])

In [5]:
errors

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time
7,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2023-03-23:20:57
8,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls –l data-store/data/input/Discovery/,ls: cannot access '–l': No such file or directory,data-store/data/input/Discovery/:\nFilteredSor...,2023-03-23:21:36
9,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls –l data-store/data/input/Discovery/TSVFiles,ls: cannot access '–l': No such file or directory,data-store/data/input/Discovery/TSVFiles:\nA_1...,2023-03-23:21:37
11,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls – l,ls: cannot access '–': No such file or directo...,NaN,2023-03-23:21:38
16,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,cd,NaN,NaN,cd RNA_SEQ_SAMPLES,/home/jovyan/.offline.sh: line 354: cd: RNA_SE...,NaN,2023-03-23:21:39
...,...,...,...,...,...,...,...,...,...,...
1510,6c940b1fbdd022d6db2d5840dfd7207c30a060b05b4353...,NaN,Bash,cd,NaN,NaN,cd gunings,/home/jovyan/.offline.sh: line 354: cd: guning...,NaN,2022-10-14:00:22
1515,6c940b1fbdd022d6db2d5840dfd7207c30a060b05b4353...,NaN,Bash,ls,NaN,Stdout.txt,ls Stdout.txt,ls: cannot access 'Stdout.txt': No such file o...,NaN,2022-10-14:00:23
1523,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2023-12-21:21:20
1530,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,ls,NaN,NaN,ls home,ls: cannot access 'home': No such file or dire...,NaN,2023-12-21:21:23


In [6]:
file_name_replacer = re.compile(r'.*((\')|(\")).+((\')|(\")).*')
file_name = re.compile(r'((\')|(\")).+((\')|(\"))')

In [7]:
# Do not parse the path name
path_replacer = re.compile(r'(\/).+(: line )(\d+).*')
path_name = re.compile(r'(\/).+(: line )(\d+)')

In [8]:
name_between_colon = re.compile(r'.+(:).+(:).+')
colon_name = re.compile(r'(:).+(:)')

In [9]:
cd_replacer = re.compile(r'.*cd:(.+):.*')
cd_name = re.compile(r'cd:(.+):')

In [10]:
def replacer(error):
    if pd.isna(error):
        return error
    if path_replacer.match(error):
        error = re.sub(path_name, 'PATH', error)
    if file_name_replacer.match(error):
        error = re.sub(file_name, '$', error)
    if name_between_colon.match(error):
        error = re.sub(colon_name, ': $:', error)
    return error;

In [11]:
df["no name"] = df["stderr"].apply(lambda x: replacer(x))
df

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time,no name
0,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:50,NaN
1,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,.,ls .,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54,NaN
2,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls,NaN,data-store\nstderr.txt\nstdout.txt\nSuperShell...,2023-03-23:20:54,NaN
3,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,..,ls ..,NaN,jovyan,2023-03-23:20:54,NaN
4,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,NaN,ls /home/jovyan/data-store/data/input/RNA_SEQ_...,NaN,Drug_A\nDrug_B,2023-03-23:20:54,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1540,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,cd,NaN,NaN,cd,NaN,NaN,2023-12-21:21:33,NaN
1541,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which pwd,NaN,/usr/bin/pwd,2023-12-21:21:36,NaN
1542,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which cd,NaN,NaN,2023-12-21:21:36,NaN
1543,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,which,NaN,NaN,which which,NaN,/usr/bin/which,2023-12-21:21:36,NaN


In [54]:
def get_command_window(full_command_list, short_command_list, error_list, window_size=6):
    command_collection = [];
    for i in range(len(full_command_list)):
        if (not pd.isna(error_list[i])) and (error_list[i] != 'None'):
            w = []
            for j in range(i, i+window_size):
                if (j >= 0 | j < len(full_command_list)):
                    if (j == i):
                        command_tuple = tuple((full_command_list[j], short_command_list[j], error_list[i]))
                    else:
                        command_tuple = tuple((full_command_list[j], short_command_list[j], 'None'))
                    w.append(command_tuple);
            
            command_collection.append(w)
    
    return command_collection

**Construct error window based on parsed error**

In [55]:
# get a list of all command history
full_command_list = df["full_command"].to_list()
command_list = df["command"].to_list()
error_list = df["no name"].to_list()

# get a list of 3 full command before and after each error
error_window = get_command_window(full_command_list, command_list, error_list)

In [46]:
error_list[:5]

[nan, nan, nan, nan, nan]

In [56]:
error_window[:5]

[[('ls Stderr.txt', 'ls', 'ls: $: No such file or directory'),
  ('ls –l data-store/data/input/Discovery/', 'ls', 'None'),
  ('ls –l data-store/data/input/Discovery/TSVFiles', 'ls', 'None'),
  ('history', 'history', 'None'),
  ('ls – l', 'ls', 'None'),
  ('ls -l', 'ls', 'None')],
 [('ls –l data-store/data/input/Discovery/',
   'ls',
   'ls: $: No such file or directory'),
  ('ls –l data-store/data/input/Discovery/TSVFiles', 'ls', 'None'),
  ('history', 'history', 'None'),
  ('ls – l', 'ls', 'None'),
  ('ls -l', 'ls', 'None'),
  ('ls -l stderr.txt', 'ls', 'None')],
 [('ls –l data-store/data/input/Discovery/TSVFiles',
   'ls',
   'ls: $: No such file or directory'),
  ('history', 'history', 'None'),
  ('ls – l', 'ls', 'None'),
  ('ls -l', 'ls', 'None'),
  ('ls -l stderr.txt', 'ls', 'None'),
  ('ls -l ./data-store/data/input/RNA_SEQ_SAMPLES/ stderr.txt', 'ls', 'None')],
 [('ls – l',
   'ls',
   'ls: $: No such file or directory\nls: $: No such file or directory'),
  ('ls -l', 'ls', 'None'

**Construct error window based on unparsed error**

In [84]:
# get a list of unparsed command history
unparsed_full_command_list = df["full_command"].to_list()
unparsed_command_list = df["command"].to_list()
unparsed_error_list = df["stderr"].to_list()

# get a list of 3 full command before and after each error
unparsed_error_window = get_command_window(unparsed_full_command_list, unparsed_command_list, unparsed_error_list)

In [85]:
unparsed_error_list[:5]

[nan, nan, nan, nan, nan]

In [86]:
unparsed_error_window[:1]

[[('ls Stderr.txt',
   'ls',
   "ls: cannot access 'Stderr.txt': No such file or directory"),
  ('ls –l data-store/data/input/Discovery/', 'ls', 'None'),
  ('ls –l data-store/data/input/Discovery/TSVFiles', 'ls', 'None'),
  ('history', 'history', 'None'),
  ('ls – l', 'ls', 'None'),
  ('ls -l', 'ls', 'None')]]

## Error Graph Algorithm

In [15]:
class Node:

    def __init__(self, program, frequency = 1, error='None') -> None:
        ''' 
        Args:
            program (String): Program that Node represents
            frequency (int): amount of times in data command proceeds parent_command
           
            command (dict): Full commands with frequencies to be used by parent nodes
            children (dict): dict of children nodes, key command, value Node
        '''
        
        self.program = program
        self.frequency = frequency
        self.error = error
        self.commands = {} # key cmd, value frequency
        self.children = {}

In [69]:
def construct_graph(error_window, error_dict={}):

    cur_node = None
    child_node = None
    
    # First loop gets one command window
    for i in range(len(error_window)):
        
        error = error_window[i][0][2]
        first_cmd = error_window[i][0][1]

        if error_dict.get(error) is None:
            cur_node = Node(program=first_cmd, error=error)
            error_dict[error] = cur_node
        else:
            cur_node = error_dict.get(error)
            cur_node.frequency += 1
        
        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, 3):
            program = error_window[i][cmd][1]
            full_program = error_window[i][cmd][0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(full_program) is None:
                child_node.commands[full_program] = 1
            else:
                child_node.commands[full_program] += 1
            
            cur_node = child_node
        
    return error_dict

KeyboardInterrupt: 

**Construct graph based on parsed error**

In [17]:
error_graph = construct_graph(error_window)

In [18]:
import math
def solution_prediction(error, graph):
    cur_node = None
    child_node = None

    if graph.get(error) is None:
        return 'Not supported';
    else:
        cur_node = graph.get(error)
        
        max_frequency = -math.inf
        next_command = ''
        for child in cur_node.children:
            child_node = cur_node.children[child]
            freq = child_node.frequency
            if freq > max_frequency:
                max_frequency = freq
                next_command = child_node.program

        return next_command

In [19]:
suggestion = solution_prediction('ls: $: No such file or directory', error_graph)
print(suggestion)

ls


In [20]:
# Accuracy
def accuracy(error_window, graph):
    correct = 0
    total = 0
    for i in range(len(error_window)):
        error = error_window[i][0][2]
        suggestion = solution_prediction(error, graph)
        if suggestion == error_window[i][3][1]:
            correct += 1
        total += 1
    return correct/total

In [21]:
acc = accuracy(error_window, error_graph)
print(acc)

0.5106382978723404


**Construct graph based on unparsed error**

In [59]:
unparsed_graph = construct_graph(unparsed_error_window)
unparsed_acc = accuracy(unparsed_error_window, unparsed_graph)
print(unparsed_acc)

0.4326241134751773


## Experiment with different window size

In [60]:
# Window size of 2
error_window = get_command_window(full_command_list, command_list, error_list, window_size=4)
error_graph = construct_graph(error_window)
acc = accuracy(error_window, error_graph)
print("accuracy with window size 5: ", acc)

# Window size of 6
error_window6 = get_command_window(full_command_list, command_list, error_list, window_size=6)
error_graph6 = construct_graph(error_window6)
acc6 = accuracy(error_window6, error_graph6)
print("accuracy with window size 6: ", acc6)

# Window size of 7
error_window7 = get_command_window(full_command_list, command_list, error_list, window_size=7)
error_graph7 = construct_graph(error_window7)
acc7 = accuracy(error_window7, error_graph7)
print("accuracy with window size 7: ", acc7)

# Window size of 8
error_window8 = get_command_window(full_command_list, command_list, error_list, window_size=8)
error_graph8 = construct_graph(error_window8)
acc8 = accuracy(error_window8, error_graph8)
print("accuracy with window size 8: ", acc8)

# Window size of 9
error_window9 = get_command_window(full_command_list, command_list, error_list, window_size=9)
error_graph9 = construct_graph(error_window9)
acc9 = accuracy(error_window9, error_graph9)
print("accuracy with window size 9: ", acc9)

accuracy with window size 5:  0.5106382978723404
accuracy with window size 6:  0.5106382978723404
accuracy with window size 7:  0.5106382978723404
accuracy with window size 8:  0.5106382978723404
accuracy with window size 9:  0.5106382978723404


## Compare ChatGPT with Graph

1. `ls Stderr` error

In [61]:
# Extract the row for ls Stderr.txt
ls_error = errors[errors['full_command'] == 'ls Stderr.txt']
ls_error

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time
7,f54d00390f36c13144c7f51475d2cf8d855ec1e20c0ac0...,emilyshen,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2023-03-23:20:57
769,094fb05870c08e0da848c72af3c1d51b412c77ce0a45de...,zwy324,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2024-01-26:18:37
1322,a9e97a896969f2a511687a9f0bf04d63501b938fee2f03...,xuanbai0110,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2023-05-24:16:46
1448,6c940b1fbdd022d6db2d5840dfd7207c30a060b05b4353...,NaN,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2022-12-01:20:08
1523,8090023b6fc6c30ba6852ec3e657289ec24344c46975fc...,johnsmith123,Bash,ls,NaN,Stderr.txt,ls Stderr.txt,ls: cannot access 'Stderr.txt': No such file o...,NaN,2023-12-21:21:20


Graph algorithm based on parsed graph

In [23]:
# The error message is ls: $: No such file or directory
suggestion = solution_prediction('ls: $: No such file or directory', error_graph)
print(suggestion)

ls


Graph algorithm based on unparsed graph

In [64]:
# Predict the suggestion
suggestion2 = solution_prediction("ls: cannot access 'Stderr.txt': No such file or directory", unparsed_graph)
print(suggestion2)

ls


ChatGPT response:

The error encountered is related to attempting to list the contents of a file named "Stderr.txt" using the ls command, resulting in a "No such file or directory" error. This issue directly corresponds to the following tutorial sections:\
**Linux: case matters**: This section would be relevant because the error demonstrates the case-sensitive nature of filenames in Linux. The command history indicates a successful listing of "stderr.txt," while the error occurred with "Stderr.txt" due to the difference in case.\
**Viewing and Moving File Nodes**: Although this section is more broadly about managing files, the specific error encountered (attempting to view a file that does not exist as named) can be addressed with knowledge from this section, especially in understanding how to accurately reference files by their correct names and paths.

To fix the error based on the exercise that requires them to execute the associated command:\
**Understand case sensitivity**: Recognize that filenames in Linux are case-sensitive. Therefore, "stderr.txt" and "Stderr.txt" are considered different filenames. The correct filename, as shown in the previous command outputs, is "stderr.txt".\
**Use the correct filename**: When attempting to list or otherwise manipulate files, ensure that the correct case is used for each part of the filename. In this case, use "ls stderr.txt" instead of "ls Stderr.txt" to avoid the "No such file or directory" error.

This solution is derived from understanding that Linux systems, including the environment where Bash commands are executed, treat filenames in a case-sensitive manner, which is a key point likely covered in the "Linux: case matters" section of the tutorial.



2. `sl` error

In [65]:
# Extract the row where command is sl
sl_error = errors[errors['full_command'] == 'sl']  
sl_error

,Log ID,Username,Log Type,command,file,filename,full_command,stderr,stdout,time
1380,6c940b1fbdd022d6db2d5840dfd7207c30a060b05b4353...,NaN,Bash,sl,NaN,NaN,sl,/home/jovyan/.offline.sh: line 368: sl: comman...,NaN,2022-12-15:19:57


In [67]:
sl_error['stderr']

1380    /home/jovyan/.offline.sh: line 368: sl: comman...
Name: stderr, dtype: object

Graph algorithm based on parsed graph

In [27]:
# The error message after parsing is: PATH: $: command not found
suggestion = solution_prediction('PATH: $: command not found', error_graph)
print(suggestion)

ls


Graph algorithm based on unparsed graph

In [68]:
suggestion3 = solution_prediction("/home/jovyan/.offline.sh: line 368: sl: command not found", unparsed_graph)
print(suggestion3)

ls


ChatGPT response:

Given the command history and the resulting error with the sl command, the tutorial sections that are most relevant to the situation include:\
**Change to arbitrary and home directory**: This section is relevant because the user utilized the cd ~ command to change the current working directory to the home directory just before attempting to execute the sl command. This demonstrates navigating the filesystem, which is a fundamental skill in Bash and command-line environments.\
**Which operation and built-in vs external application**: This error illustrates a common issue where the user attempts to use a command that is not recognized by the system, indicating it's either an external application not present on the system or a typo in a built-in command name. Since sl is an external command that needs to be installed, this section is directly relevant.\
**Execution path**: The error message implies that the shell attempted to execute a command that it could not find in the execution path. This section of the tutorial could help users understand how the shell searches for commands and why it couldn't find sl.

To fix the error and successfully run the sl command (assuming it's intended for its novelty effect of displaying an ASCII art steam locomotive across the terminal), the user needs to install the sl package. The installation process depends on the Linux distribution being used, as previously mentioned:\
On Ubuntu/Debian-based systems:\
`bash`\
`sudo apt-get update`\
`sudo apt-get install sl`

After installation, the sl command should execute successfully, displaying the locomotive animation. This correction involves recognizing the need for external applications not present by default and understanding how to install them, which is an essential skill for efficiently working within Linux environments.